In [1]:
#run this and the next block only

import os
import sys
module_path = os.path.abspath(os.path.join('../locc'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [ ]:
# this block is the visualization server.py file
import math
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import qiskit
import json

import os
import requests
import mimetypes
from flask import Flask
from flask import Response
from flask import request

import webbrowser
import threading
import time

from k_party import k_party
from entanglement_measures import EntanglementMeasures



#custom state
def custom_state_with_dims(vec,dims):
    s=0
    for x in vec: s=s+abs(x)*abs(x)#complex x
    a=np.sqrt(s)
    v=[x/a for x in vec]
    return qiskit.quantum_info.Statevector(v,dims)


graph = nx.Graph()
graph.add_nodes_from(list(range(3)))
graph.add_edges_from([(0,1),(0,2),(1,2)])

def graph_state_from_qiskit(graph):
    adj_matrix=nx.to_numpy_matrix(graph)
    circuit= qiskit.circuit.library.GraphState(adj_matrix)
    results=qiskit.quantum_info.Statevector(circuit)
    return results
    


def prepare_state_data(state_vector,name):
    dims=state_vector.dims()
    k_party_dims=[ (1,[x]) for x in dims ]
    N=max(dims)
    k_party_obj=k_party(len(dims),N,k_party_dims,state_vector)
    state_obj=[{"re":x.real,"im":x.imag} for x in state_vector]
    
    e_stats={}
    o={"state":state_obj,"name":name,"parties":len(dims),"dims":dims,
   "e_stats":e_stats}
    
    em = EntanglementMeasures(N, k_party_obj, 2)
    for i in range(len(dims)):
        for j in range(i):
            if i==j: 
                continue
            max_e=em.get_le_upper_bound(k_party_obj, i, j)
            min_e=em.get_le_lower_bound(k_party_obj, i, j)
            e_stats[str(i)+","+str(j)]={"max":max_e,"min":min_e}
    result=json.dumps(o)
    return result

def prepare_state_data_evolving(statev1,statev2,name):
    party_arr=[]
    steps=5
    dims=statev1.dims()
    k_party_dims=[ (1,[x]) for x in dims ]
    N=max(dims)
    k_party_obj=k_party(len(dims),N,k_party_dims,statev1)
    for i in range(steps):
        state_vector=custom_state_with_dims((i*statev1+(steps-i)*statev2/steps),dims)
        if state_vector.is_valid() == False: print ("Error: invalid state")
        #this normalizes the interpolated state
        k_party_obj_2=k_party(len(dims),N,k_party_dims,state_vector)
        party_arr.append(k_party_obj_2)
    state_obj=[{"re":x.real,"im":x.imag} for x in statev1]
    state_obj2=[{"re":x.real,"im":x.imag} for x in statev2]
    dims=statev1.dims()
    e_stats={}
    o={"state":state_obj,"state2":state_obj2,"name":name,"parties":len(dims),"dims":dims,
   "e_stats":e_stats,"is_changing":True,"steps":steps}
    em = EntanglementMeasures(N, k_party_obj_2, 2)
    for i in range(len(dims)):
        for j in range(i):
            if i==j: 
                continue
            max_e=em.get_le_upper_bound_evolving(party_arr, i, j)
            min_e=em.get_le_lower_bound_evolving(party_arr, i, j)
            #max_e=[(x*1+(steps-x)*(2/3))/steps for x in range(steps)]
            #min_e=[(x*0+(steps-x)*(1/3))/steps for x in range(steps)]
            e_stats[str(i)+","+str(j)]={"max":max_e,"min":min_e}
    result=json.dumps(o)
    return result

def init_states():
    for name in named_states:
        print("processing " +name)
        data=named_states[name]
        if len(data)==2:
            state=custom_state_with_dims(data[1],data[0])
            states[name]=prepare_state_data(state,name)
        else:
            state1=custom_state_with_dims(data[1],data[0])
            state2=custom_state_with_dims(data[2],data[0])
            states[name]=prepare_state_data_evolving(state1,state2,name)
        
named_states={
    "ghz":( (2,2,2), [1,0,0,0,0,0,0,1] ),
    "w":  ( (2,2,2), [0,1,1,0,1,0,0,0]),
    "epr": ( (2,2,2), [1,1,0,0,0,0,1,1] ),
    #"ghz4": ( (2,2,2,2), [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1] ),
    #"322": ( (3,2,2), [1,1,0,0,0,0,1,1,0,0,0,0] ), #error!
    #"ghz3":  ( (3,3,3),  [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]),
    #"changing":( (2,2,2), [1,0,0,0,0,0,0,1], [0,1,1,0,1,0,0,0]),
}

states={}

init_states()
state_name="ghz"
user_state=states["ghz"]



app = Flask(__name__,static_url_path='')

@app.route('/')
def index(): 
    return app.send_static_file('index.html')

@app.route('/favicon.ico')
def favicon(): 
    return app.send_static_file('favicon.ico')

@app.route('/d3.js')
def d3js(): 
    return app.send_static_file('d3.js')

@app.route('/<name>')
def index_state(name): 
    global state_name 
    global user_state
    if name in states: 
        state_name = name
        user_state=states[state_name]
        print(name, state_name)
    else:
        strs=name.split(",") #eg. each number is like 1+2j
        array=[complex(x) for x in strs]
        user_state=prepare_state_data(custom_state_from_qiskit(array),name)
        state_name=name
        states[name]=user_state
    #if state_name not in states: state_name="ghz"
        print(name, state_name,array)
    return app.send_static_file('index.html')

@app.route("/state.json")
def send_state():
    global state_name 
    global user_state
    print(state_name)
    return Response(user_state, mimetype='application/json')

def open_browser():
    time.sleep(1)
    webbrowser.open("http://localhost:3000/")

thread = threading.Thread(target=open_browser, args=())
thread.daemon = True  
thread.start() 
        

app.run(host='0.0.0.0', port=3000)



processing ghz
Optimization terminated successfully.
         Current function value: -1.000000
         Iterations: 122
         Function evaluations: 247
Entanglement entropy =  1.0
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 124
         Function evaluations: 246
Entanglement entropy =  1.1975355340017127e-20
Optimization terminated successfully.
         Current function value: -1.000000
         Iterations: 122
         Function evaluations: 254
Entanglement entropy =  1.0
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 129
         Function evaluations: 247
Entanglement entropy =  1.7080912549021341e-19
Optimization terminated successfully.
         Current function value: -1.000000
         Iterations: 106
         Function evaluations: 233
Entanglement entropy =  1.0
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 127

 * Running on all addresses.
 * Running on http://192.168.1.165:3000/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Dec/2022 19:08:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2022 19:08:18] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2022 19:08:18] "GET /state.json HTTP/1.1" 200 -


ghz


In [2]:
"ghz":prepare_state_data(ghz_state_from_qiskit(),"ghz"),
        #"w":prepare_state_data(w_state_from_qiskit(),"w"),
        #"epr":prepare_state_data(epr_state_from_qiskit(),"epr"),
        #"000":prepare_state_data(custom_state_from_qiskit([1,0,0,0,0,0,0,0]),"000"),
        #"graph":prepare_state_data(graph_state_from_qiskit(graph),"graph"),
        #"test":test_state_data(),
        "changing":prepare_state_data_evolving(ghz_state_from_qiskit(),w_state_from_qiskit(),"changing"),
        
        #"qutrits":prepare_state_data(qutrit_state_from_qiskit(),"qutrits"),
        #"qutrits":prepare_state_data(qutrit_state2_from_qiskit(),"qutrits"),
        #the above is with 3 qutrits
        #the above gives an error now
        #"qutrits":qutrit_test_state_data(),
        

def test_state_data():
    circuit = qiskit.QuantumCircuit(5)
    circuit.h(0)
    circuit.cx(0, 1)
    circuit.cx(0, 2)
    circuit.cx(0, 3)
    circuit.cx(0, 4)
    state_vector=qiskit.quantum_info.Statevector(circuit)
    
    state_obj=[{"re":x.real,"im":x.imag} for x in state_vector]
    dims=state_vector.dims()
    e_stats={}
    o={"state":state_obj,"name":"test","parties":len(dims),"dims":dims,
   "e_stats":e_stats}
    #em = EntanglementMeasures(2, tri_party, 2)
    for i in range(len(dims)):
        for j in range(i):
            if i==j: 
                continue
            #max_e=em.get_le_upper_bound(tri_party, i, j)
            #min_e=em.get_le_lower_bound(tri_party, i, j)
            max_e=2.0/3
            min_e=1.0/3
            e_stats[str(i)+","+str(j)]={"max":max_e,"min":min_e}
    result=json.dumps(o)
    return result

def qutrit_test_state_data():
    vec=[1,1,0,0,0,1,0,1,0,1,1,0]
    dims=(3,2,2)
    v=[x/a for x in vec]
    state_vector=custom_state_with_dims(vec,dims)
    state_obj=[{"re":x.real,"im":x.imag} for x in state_vector]
    #state_obj=[{"re":x,"im":0} for x in v]
    #dims=state_vector.dims()
    e_stats={}
    o={"state":state_obj,"name":"qutrits","parties":len(dims),"dims":dims,
   "e_stats":e_stats}
    #em = EntanglementMeasures(2, tri_party, 2)
    for i in range(len(dims)):
        for j in range(i):
            if i==j: 
                continue
            #max_e=em.get_le_upper_bound(tri_party, i, j)
            #min_e=em.get_le_lower_bound(tri_party, i, j)
            max_e=2.0/3
            min_e=1.0/3
            e_stats[str(i)+","+str(j)]={"max":max_e,"min":min_e}
    result=json.dumps(o)
    return result



SyntaxError: invalid syntax (<ipython-input-2-cae8c35e16d0>, line 1)

In [34]:
Error: invalid state
Error: invalid state
Error: invalid state
Error: invalid state
Error: invalid state
Error: invalid state
Error: invalid state
Error: invalid state
Error: invalid state
Error: invalid state
Error: invalid state
Error: invalid state
Error: invalid state
Error: invalid state
Error: invalid state
Error: invalid state
Error: invalid state
Error: invalid state
Error: invalid state

[1]

In [7]:

from ..locc.entanglement_measures import EntanglementMeasures
from ..locc.k_party import k_party


tri_party=k_party(3,2,[(1,[2]),(1,[2]),(1,[2])],state_vector)
em = EntanglementMeasures(2, tri_party, 2)
em.get_le_lower_bound(tri_party,0, 1)

ImportError: attempted relative import with no known parent package

In [23]:
tri_party

In [12]:
graph_state_from_qiskit(graph)

Statevector([ 0.35355339+0.j,  0.35355339+0.j,  0.35355339+0.j,
             -0.35355339+0.j,  0.35355339+0.j, -0.35355339+0.j,
             -0.35355339+0.j, -0.35355339+0.j],
            dims=(2, 2, 2))


In [32]:
(graph_state_from_qiskit(graph)+ghz_state_from_qiskit())/2

Statevector([ 0.53033009+0.j,  0.1767767 +0.j,  0.1767767 +0.j,
             -0.1767767 +0.j,  0.1767767 +0.j, -0.1767767 +0.j,
             -0.1767767 +0.j,  0.1767767 +0.j],
            dims=(2, 2, 2))


In [13]:
s=(graph_state_from_qiskit(graph)+ghz_state_from_qiskit())/2

In [17]:
s

Statevector([ 0.53033009+0.j,  0.1767767 +0.j,  0.1767767 +0.j,
             -0.1767767 +0.j,  0.1767767 +0.j, -0.1767767 +0.j,
             -0.1767767 +0.j,  0.1767767 +0.j],
            dims=(2, 2, 2))


In [16]:
custom_state_from_qiskit(s)

Statevector([ 0.375+0.j,  0.125+0.j,  0.125+0.j, -0.125+0.j,  0.125+0.j,
             -0.125+0.j, -0.125+0.j,  0.125+0.j],
            dims=(2, 2, 2))


In [30]:
state_vector+state_vector

NameError: name 'state_vector' is not defined

In [83]:
np.reshape(state_vector,(2,2,2))

array([[[0.        +0.j, 0.57735027+0.j],
        [0.57735027+0.j, 0.        +0.j]],

       [[0.57735027+0.j, 0.        +0.j],
        [0.        +0.j, 0.        +0.j]]])

In [16]:
def GHZ(dims):
    psi = np.zeros((dims, dims, dims))

    np.fill_diagonal(psi, np.sqrt(1/dims))
    ghz = qiskit.quantum_info.Statevector(psi.flatten())

    print(ghz)
    return psi
ghz=GHZ(2)
ghzk=k_party(3,2,[(1,[2]),(1,[2]),(1,[2])],ghz)
em = EntanglementMeasures(2, GHZ(2), 2)
em.get_le_upper_bound(ghzk,1, 2)

Statevector([0.70710678+0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.70710678+0.j],
            dims=(2, 2, 2))
Statevector([0.70710678+0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.70710678+0.j],
            dims=(2, 2, 2))
Optimization terminated successfully.
         Current function value: -0.000000
         Iterations: 80
         Function evaluations: 219
Entanglement entropy =  6.406853007629843e-16


6.406853007629843e-16

In [16]:
em.get_le_lower_bound(np.reshape(state_vector,(2,2,2)), 0, 1)

Entanglement entropy =  0.0006812896252176074


0.0006812896252176074

In [26]:
tri_party=k_party(3,[(1,[2]),(1,[2]),(1,[2])],state_vector)
em = EntanglementMeasures(2, tri_party, 2)
em.get_le_lower_bound(tri_party,0, 1)

AttributeError: 'k_party' object has no attribute 'flatten'

In [27]:
from k_party import k_party

k=k_party(3,2,[(1,[2]),(1,[2]),(1,[2])],state_vector)

TypeError: __init__() takes 4 positional arguments but 5 were given

In [ ]:
em.get_le_upper_bound(np.reshape(state_vector,(2,2,2)), 0, 1)

In [63]:
for i in range(len(dims)): 
    print(i)

0
1
2


In [60]:
range(len(dims)

3

In [12]:
from qiskit import QuantumCircuit
from qiskit import execute, IBMQ, BasicAer

def w_state_circuit_from_qiskit():
    a=np.sqrt(1/3)
    v=[0,a,a,0,a,0,0,0]
    #v=[1,0,0,0,0,0,0,0]
    circuit = QuantumCircuit(3,3)
    circuit.initialize(v, circuit.qubits)
    circuit.draw("text")
    return circuit
    
#GHZ state
def ghz_state_circuit_from_qiskit():
    circuit = qiskit.QuantumCircuit(3,3)
    circuit.h(0)
    circuit.cx(0, 1)
    circuit.cx(0, 2)
    circuit.draw("text")
    return circuit

#GHZ state 2
def ghz_state_circuit_from_qiskit2():
    psi = np.zeros((2, 2, 2))
    np.fill_diagonal(psi, np.sqrt(1/2))
    circuit = qiskit.QuantumCircuit(3,3)
    circuit.initialize(psi.flatten(), circuit.qubits)
    return  circuit

qc=ghz_state_circuit_from_qiskit2()#w_state_circuit_from_qiskit()
qc.measure([0, 1, 2], [0, 1, 2])

# See a list of available local simulators
print("BasicAer backends: ", BasicAer.backends())
backend_sim = BasicAer.get_backend('qasm_simulator')

# Compile and run the Quantum circuit on a simulator backend
job_sim = execute(qc, backend_sim)
result_sim = job_sim.result()

# Show the results
print(result_sim.get_counts(qc))




BasicAer backends:  [<QasmSimulatorPy('qasm_simulator')>, <StatevectorSimulatorPy('statevector_simulator')>, <UnitarySimulatorPy('unitary_simulator')>]
{'000': 507, '111': 517}


In [13]:
#GHZ state 2
def ghz_state_circuit_from_qiskit2():
    psi = np.zeros((2, 2, 2))
    np.fill_diagonal(psi, np.sqrt(1/2))
    circuit = qiskit.QuantumCircuit(3,3)
    circuit.initialize(psi.flatten(), circuit.qubits)
    return  circuit
ghz_state_circuit_from_qiskit2()

In [6]:
def graph_state_from_qiskit(graph):
    #print graph circuit
    adj_matrix=nx.to_numpy_matrix(graph)#adjacency_matrix(graph)
    circuit= qiskit.circuit.library.GraphState(adj_matrix)
    circuit.draw("text")
    
    results=qiskit.quantum_info.Statevector(circuit)
    
    return results

state_vector=graph_state_from_qiskit(graph)
state_obj=[{"re":x.real,"im":x.imag} for x in state_vector]
json.dumps(state_obj)

'[{"re": 0.3535533905932737, "im": 0.0}, {"re": 0.3535533905932737, "im": 0.0}, {"re": 0.3535533905932737, "im": 0.0}, {"re": -0.3535533905932737, "im": 0.0}, {"re": 0.3535533905932737, "im": 0.0}, {"re": 0.3535533905932737, "im": 0.0}, {"re": -0.3535533905932737, "im": 0.0}, {"re": 0.3535533905932737, "im": 0.0}]'

In [17]:
from flask import Flask

app = Flask(__name__,static_url_path='')


@app.route('/')
def index(): 
    return app.send_static_file('index.html')

@app.route("/state.json")
def send_state():
    state_obj=[{"re":x.real,"im":x.imag} for x in state_vector]
    dims=state_vector.dims()
    o={"state":state_obj,"parties":len(dims),"dims":dims}
    result=json.dumps(o)
    return(result)

app.run(host='0.0.0.0', port=3000)


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.1.165:3000/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Nov/2022 19:52:59] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [11/Nov/2022 19:53:11] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [11/Nov/2022 19:53:24] "GET /state.json HTTP/1.1" 200 -
127.0.0.1 - - [11/Nov/2022 19:54:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Nov/2022 19:54:23] "GET /d3.js HTTP/1.1" 200 -


In [9]:


class MyHTTPRequestHandler(BaseHTTPRequestHandler):
    
    def do_GET(self):
        filepath = self.path
        #queries = urlparse.parse_qs(urlparse.urlparse(self.path).query)
        #.get('imsi', None)  None or the string value of imsi
        if filepath=="/":
            filepath="/index.html"
        if filepath=="/test.json":
            try:
                self.send_response(200)
                self.send_header('Content-type', 'application/json')
                self.end_headers();
                state_obj=[{"re":x.real,"im":x.imag} for x in state_vector]
                dims=state_vector.dims()
                o={"state":state_obj,"parties":len(dims),"dims":dims}
                result=json.dumps(o)
                self.wfile.write(bytes(result, 'UTF-8'))
                return
            except IOError:
                self.send_error(404,'File Not Found: %s ' % filepath)
        try:
            print("opening"+os.path.join('.',  filepath))
            f = open(os.path.join('./',  filepath))#'www',
        except IOError:
            self.send_error(404,'File Not Found: %s ' % filepath)
        else:
            self.send_response(200)
            mimetype, _ = mimetypes.guess_type(filepath)
            self.send_header('Content-type', mimetype)
            self.end_headers()
            for s in f:
                self.wfile.write(s)

    def do_POST(self):
        
        length = int(self.headers["Content-Length"])
        print("Data: " + str(self.rfile.read(length), "utf-8"))

        response = bytes("This is the response.", "utf-8") #create response

        self.send_response(200) #create header
        self.send_header("Content-Length", str(len(response)))
        self.end_headers()

        self.wfile.write(response) #send response





def run(server_class=HTTPServer, handler_class=MyHTTPRequestHandler):
    server_address = ('', 8001)
    httpd = server_class(server_address, handler_class)
    try:
        httpd.serve_forever()
    except Error:
        print(Error)
        pass
    httpd.server_close()

run()

OSError: [Errno 98] Address already in use